In [4]:
from commonfunctions import *
import numpy as np
import skimage.io as io
from skimage.exposure import equalize_hist
from skimage.filters import *
from skimage.measure import find_contours
from skimage.morphology import *
from skimage.transform import *
from skimage.util import *
from skimage import data
import matplotlib.pyplot as plt
import os


In [5]:
def binarize(img):
    if len(img.shape) > 2 :
        img = (rgb2gray(img)*255).astype("uint8")
    #thresh_local = threshold_local(img, 31, offset=3)
    thresh_sauvola = threshold_sauvola(img, 31)
    return img >= thresh_sauvola

def get_rotation_angle(img):
    # using hough transform to measure the angle
    img = invert(img)
    tested_angles = np.linspace(-np.pi / 2, np.pi / 2, 360)
    h, theta, d = hough_line(img, theta=tested_angles)
    ind = np.unravel_index(np.argmax(h, axis=None), h.shape)
    degree = np.rad2deg(theta[ind[1]])
    return degree

def fix_rotation(img, angle):
    if angle > 0:
        img = rotate(img, -(90-angle))
    else:
        img = rotate(img, 90 - abs(angle))
    return img


In [9]:
input_imgs = []
output_imgs = []
temp = io.imread("input/handwritted2.png")
temp  = binarize([img] , ["temp"])
show_images(temp)
for filename in os.listdir("input"):
    img = io.imread("input/" + filename)
    input_imgs.append(img)
    img = binarize(img)
    img = fix_rotation(img, get_rotation_angle(img))
    io.imsave("output/" + os.path.splitext(filename)[0] + ".png", img)
    output_imgs.append(img)

show_images(input_imgs)
show_images(output_imgs)

TypeError: binarize() takes 1 positional argument but 2 were given